# load
FIle: ECL Computation received by Trade Finance with as of Aug 2024 send by Fariza Syafiqah Binti Edison (fsyafiqahedison@exim.com.my)

In [1]:
date_file = 202408

T = "ECL computation Aug 2024 (Trade)"

prev_file = "Jul"

#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(date_file)+"\\Trade"

T1 = pd.read_excel(str(Location)+"\\"+str(T)+".xlsm", sheet_name='Summary', header=6)

LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(date_file-1)+"\\Loan Database as "+str(prev_file)+" "+str(date_file)[:4]+" (python) - Final.xlsx", sheet_name='Export Worksheet', header=1) # skiprows = 2, nrows=18,  usecols= 'A:D'

MRate = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(date_file)+"\\Working\\00. Month End Rate.xlsx",sheet_name="Sheet1", header=2) # skiprows = 2, nrows=18,  usecols= 'A:D'


# Process

In [2]:
T1.columns = T1.columns.str.strip()
T1['Account No'] = T1['Account No'].astype(str)

LDB_prev.columns = LDB_prev.columns.str.strip()
LDB_prev.iloc[np.where(LDB_prev['EXIM Account No.']!="Total")]
LDB_prev['Finance(SAP) Number'] = LDB_prev['Finance(SAP) Number'].astype(str)

T2 = T1.iloc[np.where(~T1.Currency.isna())][['Account No','Borrower name','Undrawn amount (base currency)','Profit Rate/ EIR','Currency',
          'First Released Date','Maturity date','Availability period','DPD','Principal payment frequency','Interest payment frequency']].rename(columns={'Account No':'Finance(SAP) Number'})

merge = T2.merge(LDB_prev[['CIF Number','EXIM Account No.','Finance(SAP) Number']], on='Finance(SAP) Number', how='left')

merge1 = merge.merge(MRate[['Month','Curr']].rename(columns={'Month':'Currency'}), on='Currency', how='left')

merge1['Unutilised/ Undrawn Amount (MYR)'] = merge1['Undrawn amount (base currency)']*merge1['Curr']
merge1['Profit Rate/ EIR'] = merge1['Profit Rate/ EIR']*100

merge1 = merge1.rename(columns={'Undrawn amount (base currency)':'Unutilised/ Undrawn Amount (Facility Currency)',
                               'Profit Rate/ EIR':'Average Profit/Interest Rate'})

merge1 = merge1[[ 'CIF Number','EXIM Account No.','Finance(SAP) Number', 'Borrower name', 'Currency', 'Curr',
       'Unutilised/ Undrawn Amount (Facility Currency)', 'Unutilised/ Undrawn Amount (MYR)',
       'Average Profit/Interest Rate',
          'First Released Date','Maturity date','Availability period','DPD','Principal payment frequency','Interest payment frequency']]

In [3]:
merge1.dtypes

CIF Number                                                object
EXIM Account No.                                          object
Finance(SAP) Number                                       object
Borrower name                                             object
Currency                                                  object
Curr                                                     float64
Unutilised/ Undrawn Amount (Facility Currency)           float64
Unutilised/ Undrawn Amount (MYR)                         float64
Average Profit/Interest Rate                             float64
First Released Date                               datetime64[ns]
Maturity date                                             object
Availability period                               datetime64[ns]
DPD                                                      float64
Principal payment frequency                               object
Interest payment frequency                                object
dtype: object

In [4]:
import datetime
merge1['Maturity date'] = pd.to_datetime(merge1['Maturity date'], format='%Y%m%d')

# Export

In [5]:
T[-16:-8] #= "05. ECL Computation Client Template June-24 (Regular)"

'Aug 2024'

In [6]:
merge1.to_excel(r"C:\\Users\\syahidhalid\\Syahid_PC\\Analytics - FAD\\00. Automation LDB\\Data Source\\"+str(date_file)+"\\Working\\07. Undrawn Trade "+str(T[-16:-8])+".xlsx", sheet_name='Export Worksheet', index=False)

In [47]:
T2.shape

(40, 11)

In [46]:
merge.shape

(40, 13)